[프로세스]

1. 하나의 tourney 대상으로 메타 분석
    play.limitlesstcg.com/tournament/6779bb0bed2c78098c6b086b/metagame

    step a) 아키타입 별 상대 승률
    step b) 20장 덱 유형 별 vs 아키타입 승률
    step c) 아키타입 별 승률 높은 20장 덱 유형 분석 (궁극의 덱이 무엇일까)

2. 전체 tourney 묶어서 다시 분석

    step a) Standard한 Tourney 분류
            포맷을 Standard로 설정해도, 스탠다드 룰이 아닌 Tourney가 있음. 
            

# Base

In [1]:
### 라이브러리 임포트

import pandas as pd
import re
import time
import requests
from datetime import datetime
from datetime import date
import pandas as pd
from bs4 import BeautifulSoup
import json

In [82]:
### 헤더, 함수 정의

headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36",
        "Connection": "keep-alive",
        "Cache-Control": "max-age=0",
        "sec-ch-ua-mobile": "?0",
        "DNT": "1",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
        "Sec-Fetch-Site": "none",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-User": "?1",
        "Sec-Fetch-Dest": "document",
        "Accept-Encoding": "gzip, deflate, br",
        "Accept-Language": "ko-KR,ko;q=0.9"
    }

def get_soup(url, time_sleep=0, max_retries=600):
    try:
        if max_retries <= 0:
            print("[최대 재시도 횟수 초과 : get_soup()]")
            return None
        with requests.Session() as session:
            response = session.get(url, headers=headers)
            time.sleep(time_sleep)
        soup = BeautifulSoup(response.text, "html.parser")
        if len(soup) == 0:  # 불러오는데 실패하면
            print(f"[soup 로딩 실패, 반복] [get_soup(time_sleep=1)]")
            soup = get_soup(url, 1)
    except Exception as e:
        print(f"[오류 발생, 반복] [get_soup(time_sleep=1)] ", e)
        soup = get_soup(url, 1, max_retries-1)
    return soup

In [116]:
# 함수 2
# 전처리 함수 : 덱 타입
def preprocess_deck_type(deck_type):
    processed_deck_type = re.sub(r'[^a-zA-Z ]', '', deck_type)  # 영어 알파벳(대소문자)과 공백만 남기고 모두 제거
    return processed_deck_type.replace(' ', '_')  # 공백문자를 _로 변경

# 시간측정 데코레이터
def time_check(func):
  def wrapper(*args, **kwargs):
    start_time = datetime.now().replace(microsecond=0)  # 시작 시각
    print(f"함수 {func.__name__} 실행 시작 시간: {start_time}")
    
    result = func(*args, **kwargs) # 함수 실행

    end_time = datetime.now().replace(microsecond=0)  # 종료 시각
    print(f"함수 {func.__name__} 실행 종료 시간: {end_time}")
    print(f"함수 {func.__name__} 실행 시간: {end_time - start_time}")

    return result
  return wrapper

# tourney_data 에 1개의 tourney 메타 정보 누적
def crawl_one_tourney_meta(tourney_url, tourney_data, str_progress):
    tourney_data_copy = tourney_data
    print("[start] crawl_one_tourney_meta()")
    tourney_meta_url = tourney_url + "/metagame"
    limitlesstcg_url_base = "https://play.limitlesstcg.com"
    soup = get_soup(tourney_meta_url)    # bs4로 html 가져오기
    tourney_deck_type_list = []
    min_deck_count = 1              # 최소 덱 타입 수

    # 정보 추출 : 덱 타입
    meta_tr_list = soup.select('table.meta tr')[1:]
    for tr in meta_tr_list:
        deck_count = int(tr.select('td')[1].text)
        if deck_count < min_deck_count:
            continue
        deck_type = preprocess_deck_type(tr.select('td')[2].text)
        deck_url = limitlesstcg_url_base + tr.select('td')[2].find('a')['href']
        # print(deck_count, deck_type, deck_url)
    
        # DB에 덱타입 추가 : 기존 정보가 있는지 확인
        if deck_type not in tourney_data_copy:
            tourney_data_copy[deck_type] = {}
        
        # df에 저장
        row = [deck_type, deck_count, deck_url]
        tourney_deck_type_list.append(row)
    
    # df : 덱 타입 (중복 덱타입 존재함)
    df_tourney_deck_type_list  = pd.DataFrame(tourney_deck_type_list, columns=["deck_type", "deck_count", "deck_url"])

    # 덱 타입 별 승/패 수 누적
    for index, row in df_tourney_deck_type_list.iterrows():
        print(f"Tourney {str_progress} Deck Type [{index+1}/{len(df_tourney_deck_type_list)}] {row['deck_type']}")
        matchup_deck_type = row['deck_type']
        matchup_url = row['deck_url'] + "/matchups"
        matchup_soup = get_soup(matchup_url)
    
        ### 정보 추출 : 덱 별 매치업 통계
        matchup_tr_list = matchup_soup.select('table.striped tr')[1:-1]
        
        for tr in matchup_tr_list:
            # 정보 추출
            opponent_deck_type = preprocess_deck_type(tr.select('td')[1].text)
            win_count = int(tr.select('td')[2].text.split(' - ')[0])
            lose_count = int(tr.select('td')[2].text.split(' - ')[1])
            # print(opponent_deck_type, win_count, lose_count)
        
            # 정보 저장 : opponent_deck_type이 tourney_data[deck_type]에 있는지 확인 (없으면 추가)
            try:
                tourney_data_copy[matchup_deck_type][opponent_deck_type]
            except: 
                tourney_data_copy[matchup_deck_type][opponent_deck_type] = {"win_count": 0, "lose_count": 0}
        
            # 3. 승리/패배 횟수 누적
            tourney_data_copy[matchup_deck_type][opponent_deck_type]["win_count"] += win_count
            tourney_data_copy[matchup_deck_type][opponent_deck_type]["lose_count"] += lose_count
    
            # print(tourney_data[matchup_deck_type])
    
    print("[end] crawl_one_tourney_meta()")
    
    return tourney_data_copy

@time_check
def crawl_tourney_meta(tourney_result_file_name):
    # 1. 파일 불러오기
    df = pd.read_csv("./tourney_result/" + tourney_result_file_name, encoding='utf-8')
    df.info()

    # tourney_meta 선언
    tourney_meta = {}

    # 2. 모든 url에 대해 meta 정보 수집
    for index, row in df.iterrows():
        str_progress = f"[{index+1}/{len(df)}]"
        print(f"Tourney Meta {str_progress} {row['url']}")
        tourney_meta = crawl_one_tourney_meta(row['url'], tourney_meta, str_progress)

    # JSON 파일로 저장 
    meta_file_name = tourney_result_file_name.replace('csv', 'json').replace('result', 'meta')
    with open("./tourney_meta/" + meta_file_name, "w") as f:
        json.dump(tourney_meta, f, indent=4)  # indent=4는 가독성을 위해 들여쓰기


In [89]:
### 파일 불러오기
tourney_result_file_name = "tourney_result_250125.csv"
df = pd.read_csv("./tourney_result/" + tourney_result_file_name, encoding='utf-8')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126 entries, 0 to 125
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   date          126 non-null    object
 1   tourney_name  126 non-null    object
 2   organizer     126 non-null    object
 3   players       126 non-null    int64 
 4   winner        126 non-null    object
 5   country       126 non-null    object
 6   url           126 non-null    object
dtypes: int64(1), object(6)
memory usage: 7.0+ KB


In [117]:
t_data = crawl_one_tourney_meta("https://play.limitlesstcg.com/tournament/678305977960ec710eff1898", {}, "d")
t_data

[start] crawl_one_tourney_meta()
Tourney d Deck Type [1/168] Mewtwo_ex_Gardevoir
Tourney d Deck Type [2/168] Pikachu_ex_Zebstrika
Tourney d Deck Type [3/168] Gyarados_ex_Greninja
Tourney d Deck Type [4/168] Exeggutor_ex_Celebi_ex
Tourney d Deck Type [5/168] Charizard_ex_Moltres_ex
Tourney d Deck Type [6/168] Celebi_ex_Serperior
Tourney d Deck Type [7/168] Charizard_ex_Arcanine_ex
Tourney d Deck Type [8/168] Aerodactyl_ex_Primeape
Tourney d Deck Type [9/168] Arcanine_ex_Moltres_ex
Tourney d Deck Type [10/168] Starmie_ex_Lumineon
Tourney d Deck Type [11/168] Greninja_Mew_ex
Tourney d Deck Type [12/168] Greninja_Farfetchd
Tourney d Deck Type [13/168] Scolipede_Weezing
Tourney d Deck Type [14/168] Aerodactyl_ex_Farfetchd
Tourney d Deck Type [15/168] Pidgeot_ex_Pidgeot
Tourney d Deck Type [16/168] Pikachu_ex_Raichu
Tourney d Deck Type [17/168] Golem_Druddigon
Tourney d Deck Type [18/168] Aerodactyl_ex_Mew_ex
Tourney d Deck Type [19/168] Starmie_ex_Articuno_ex
Tourney d Deck Type [20/168] Pi

{'Mewtwo_ex_Gardevoir': {'Blastoise_ex_Starmie_ex': {'win_count': 2,
   'lose_count': 0},
  'Alakazam_Mew_ex': {'win_count': 2, 'lose_count': 0},
  'Zebstrika_Pikachu_ex': {'win_count': 2, 'lose_count': 0},
  'Wigglytuff_ex_Exeggutor_ex': {'win_count': 2, 'lose_count': 0},
  'Dragonite_Zebstrika': {'win_count': 2, 'lose_count': 0},
  'Machamp_ex_Aerodactyl_ex': {'win_count': 2, 'lose_count': 0},
  'Mewtwo_ex_Alakazam': {'win_count': 2, 'lose_count': 0},
  'Gyarados_ex_Kabutops': {'win_count': 2, 'lose_count': 0},
  'Greninja_Hitmonlee': {'win_count': 1, 'lose_count': 0},
  'Golbat_Mew_ex': {'win_count': 1, 'lose_count': 0},
  'Marowak_ex_Sandslash': {'win_count': 1, 'lose_count': 0},
  'Beheeyem_Mew_ex': {'win_count': 1, 'lose_count': 0},
  'Moltres_ex_Rapidash': {'win_count': 1, 'lose_count': 0},
  'Arcanine_ex_Salazzle': {'win_count': 1, 'lose_count': 0},
  'Machamp_ex_Druddigon': {'win_count': 1, 'lose_count': 0},
  'Alakazam_Jynx': {'win_count': 1, 'lose_count': 0},
  'Golem_Primea

# 1. 실험 : 하나의 tourney 대상으로 메타 분석

In [3]:
### 저장할 데이터베이스 초기화 : 2중 딕셔너리로 저장할 예정
tourney_data = {}

In [4]:
### 샘플 토너먼트 url 설정
# tourney_meta_url = "https://play.limitlesstcg.com/tournament/6779bb0bed2c78098c6b086b/metagame"
tourney_meta_url = "https://play.limitlesstcg.com/tournament/678305977960ec710eff1898/metagame"
limitlesstcg_url_base = "https://play.limitlesstcg.com"

## step a) 아키타입 별 상대 승률

In [22]:
### 전처리 함수 : 덱 타입
def preprocess_deck_type(deck_type):
    processed_deck_type = re.sub(r'[^a-zA-Z ]', '', deck_type)  # 영어 알파벳(대소문자)과 공백만 남기고 모두 제거
    return processed_deck_type.replace(' ', '_')  # 공백문자를 _로 변경

### bs4로 html 가져오기
soup = get_soup(tourney_meta_url)

### 정보 추출 : 덱 타입
tourney_deck_type_list = []
min_deck_count = 1  # 최소 덱 타입 수

meta_tr_list = soup.select('table.meta tr')
for tr in meta_tr_list[1:]:
    # 정보 추출
    deck_count = int(tr.select('td')[1].text)
    if deck_count < min_deck_count:
        continue
    deck_type = preprocess_deck_type(tr.select('td')[2].text)
    deck_url = limitlesstcg_url_base + tr.select('td')[2].find('a')['href']
    print(deck_count, deck_type, deck_url)

    # DB에 덱타입 추가 : 기존 정보가 있는지 확인
    if deck_type not in tourney_data:
        tourney_data[deck_type] = {}
    
    # df에 저장
    row = [deck_type, deck_count, deck_url]
    tourney_deck_type_list.append(row)

# df : 덱 타입 - 중복 덱타입 존재함.
df_tourney_deck_type_list  = pd.DataFrame(tourney_deck_type_list, columns=["deck_type", "deck_count", "deck_url"])

328 mewtwo_ex_gardevoir https://play.limitlesstcg.com/tournament/678305977960ec710eff1898/metagame/mewtwo-ex-gardevoir-a1
237 pikachu_ex_zebstrika https://play.limitlesstcg.com/tournament/678305977960ec710eff1898/metagame/pikachu-ex-zebstrika-a1
175 gyarados_ex_greninja https://play.limitlesstcg.com/tournament/678305977960ec710eff1898/metagame/gyarados-ex-a1a-greninja-a1
103 exeggutor_ex_celebi_ex https://play.limitlesstcg.com/tournament/678305977960ec710eff1898/metagame/exeggutor-ex-a1-celebi-ex-a1a
73 charizard_ex_moltres_ex https://play.limitlesstcg.com/tournament/678305977960ec710eff1898/metagame/charizard-ex-moltres-ex-a1
63 celebi_ex_serperior https://play.limitlesstcg.com/tournament/678305977960ec710eff1898/metagame/celebi-ex-serperior-a1a
52 charizard_ex_arcanine_ex https://play.limitlesstcg.com/tournament/678305977960ec710eff1898/metagame/charizard-ex-arcanine-ex-a1
35 aerodactyl_ex_primeape https://play.limitlesstcg.com/tournament/678305977960ec710eff1898/metagame/aerodactyl-

In [336]:
len(tourney_data)

251

In [337]:
len(tourney_deck_type_list)  
df_tourney_deck_type_list

,deck_type,deck_count,deck_url
0,mewtwo_ex_gardevoir,328,https://play.limitlesstcg.com/tournament/67830...
1,pikachu_ex_zebstrika,237,https://play.limitlesstcg.com/tournament/67830...
2,gyarados_ex_greninja,175,https://play.limitlesstcg.com/tournament/67830...
3,exeggutor_ex_celebi_ex,103,https://play.limitlesstcg.com/tournament/67830...
4,charizard_ex_moltres_ex,73,https://play.limitlesstcg.com/tournament/67830...
...,...,...,...
163,aerodactyl_ex_weezing,1,https://play.limitlesstcg.com/tournament/67830...
164,primeape_kabutops,1,https://play.limitlesstcg.com/tournament/67830...
165,moltres_ex_volcarona,1,https://play.limitlesstcg.com/tournament/67830...
166,zapdos_ex_raichu,1,https://play.limitlesstcg.com/tournament/67830...


In [318]:
### 덱 타입 필터링 
# min_deck_count = 300
# df_tourney_deck_type_filtered = df_tourney_deck_type_list[df_tourney_deck_type_list['deck_count'] >= min_deck_count]
# df_tourney_deck_type_filtered.tail()

,deck_type,deck_count,deck_url
0,mewtwo_ex_gardevoir,302,https://play.limitlesstcg.com/tournament/6779b...


In [338]:
### 덱 타입 별 

for index, row in df_tourney_deck_type_list.iterrows():
    print(f"index : {index}")
    print(row['deck_type'])
    matchup_deck_type = row['deck_type']
    matchup_url = row['deck_url'] + "/matchups"
    matchup_soup = get_soup(matchup_url)

    ### 정보 추출 : 덱 별 매치업 통계
    matchup_tr_list = matchup_soup.select('table.striped tr')[1:-1]
    
    for tr in matchup_tr_list:
        # 정보 추출
        opponent_deck_type = tr.select('td')[1].text.replace(' ', '_')
        win_count = int(tr.select('td')[2].text.split(' - ')[0])
        lose_count = int(tr.select('td')[2].text.split(' - ')[1])
        print(opponent_deck_type, win_count, lose_count)
    
        # 정보 저장 : opponent_deck_type이 tourney_data[deck_type]에 있는지 확인 (없으면 추가)
        try:
            tourney_data[matchup_deck_type][opponent_deck_type]
        except: 
            tourney_data[matchup_deck_type][opponent_deck_type] = {"win_count": 0, "lose_count": 0}
    
        # 3. 승리/패배 횟수 누적
        tourney_data[matchup_deck_type][opponent_deck_type]["win_count"] += win_count
        tourney_data[matchup_deck_type][opponent_deck_type]["lose_count"] += lose_count

        # print(tourney_data[matchup_deck_type])

print("[done] ")

index : 0
mewtwo_ex_gardevoir
Blastoise_ex_Starmie_ex 2 0
Alakazam_Mew_ex 2 0
Zebstrika_Pikachu_ex 2 0
Wigglytuff_ex_Exeggutor_ex 2 0
Dragonite_Zebstrika 2 0
Machamp_ex_Aerodactyl_ex 2 0
Mewtwo_ex_Alakazam 2 0
Gyarados_ex_Kabutops 2 0
Greninja_Hitmonlee 1 0
Golbat_Mew_ex 1 0
Marowak_ex_Sandslash 1 0
Beheeyem_Mew_ex 1 0
Moltres_ex_Rapidash 1 0
Arcanine_ex_Salazzle 1 0
Machamp_ex_Druddigon 1 0
Alakazam_Jynx 1 0
Golem_Primeape 1 0
Gardevoir_Mewtwo 1 0
Gengar_ex_Druddigon 1 0
Alakazam_Clefable 1 0
Greninja_Melmetal 1 0
Golem_Farfetch'd 1 0
Aerodactyl_ex_Primeape 1 0
Mew_ex_Greninja 1 0
Machamp_ex_Mew_ex 1 0
Raichu_Zebstrika 1 0
Mew_ex_Golem 1 0
Alakazam_Florges 1 0
Machamp_ex_Primeape 1 0
Wigglytuff_ex_Weezing 1 0
Gyarados_ex_Chatot 1 0
Arcanine_ex_Greninja 1 0
Exeggutor_ex_Butterfree 1 0
Moltres_ex_Volcarona 1 0
Blastoise_ex_Druddigon 1 0
Machamp_ex_Hitmonlee 1 0
Golem_Druddigon 14 2
Pidgeot_ex_Hitmonlee 5 1
Ninetales_Rapidash 4 1
Charizard_ex_Moltres_ex 44 17
Aerodactyl_ex_Primeape 24 10

In [339]:
tourney_data

{'mewtwo_ex_gardevoir': {'Golem': {'win_count': 7, 'lose_count': 2},
  'Alakazam_Beheeyem': {'win_count': 4, 'lose_count': 0},
  'Exeggutor_ex_Greninja': {'win_count': 4, 'lose_count': 2},
  'Pidgeot_ex_Hitmonlee': {'win_count': 7, 'lose_count': 1},
  'Aerodactyl_ex_Onix': {'win_count': 3, 'lose_count': 1},
  'Articuno_ex_Mew_ex': {'win_count': 2, 'lose_count': 0},
  "Mew_ex_Farfetch'd": {'win_count': 4, 'lose_count': 1},
  'Greninja_Marshadow': {'win_count': 2, 'lose_count': 0},
  'Mew_ex_Alakazam': {'win_count': 2, 'lose_count': 0},
  'Moltres_ex_Rapidash': {'win_count': 3, 'lose_count': 0},
  'Marowak_ex_Aerodactyl_ex': {'win_count': 1, 'lose_count': 1},
  'Marowak_ex_Primeape': {'win_count': 2, 'lose_count': 0},
  'Marowak_ex_Golem': {'win_count': 1, 'lose_count': 0},
  "Golem_Farfetch'd": {'win_count': 2, 'lose_count': 0},
  'Machamp_ex_Mew_ex': {'win_count': 2, 'lose_count': 0},
  'Pidgeot_ex_Greninja': {'win_count': 1, 'lose_count': 0},
  'Mew_ex_Mewtwo_ex': {'win_count': 1, 'lo

In [328]:
# ### 테스트용 (위에 덧댈거임)
# matchup_deck_type = 'Mewtwo_ex_Gardevoir'
# matchup_url = "https://play.limitlesstcg.com/tournament/6779bb0bed2c78098c6b086b/metagame/mewtwo-ex-gardevoir-a1" + "/matchups"
# matchup_soup = get_soup(matchup_url)

# ### 정보 추출 : 덱 별 매치업 통계
# tr_list = soup.select('table.striped tr')[1:-1]

# for tr in tr_list:
#     # 정보 추출
#     opponent_deck_type = tr.select('td')[1].text.replace(' ', '_')
#     win_count = int(tr.select('td')[2].text.split(' - ')[0])
#     lose_count = int(tr.select('td')[2].text.split(' - ')[1])
#     print(opponent_deck_type, win_count, lose_count)

#     # 정보 저장 : opponent_deck_type이 tourney_data[deck_type]에 있는지 확인 (없으면 추가)
#     if opponent_deck_type not in tourney_data[deck_type][opponent_deck_type]:
#         tourney_data[matchup_deck_type][opponent_deck_type] = {"승리": 0, "패배": 0}

#     # 3. 승리/패배 횟수 누적
#     tourney_data[matchup_deck_type][opponent_deck_type]["승리"] += win_count
#     tourney_data[matchup_deck_type][opponent_deck_type]["패배"] += lose_count

In [331]:
backup_tourney_data = tourney_data
backup_tourney_data

{'mewtwo_ex_gardevoir': {'Golem': {'win_count': 4, 'lose_count': 0},
  'Alakazam_Beheeyem': {'win_count': 4, 'lose_count': 0},
  'Exeggutor_ex_Greninja': {'win_count': 3, 'lose_count': 0},
  'Pidgeot_ex_Hitmonlee': {'win_count': 2, 'lose_count': 0},
  'Aerodactyl_ex_Onix': {'win_count': 2, 'lose_count': 0},
  'Articuno_ex_Mew_ex': {'win_count': 2, 'lose_count': 0},
  "Mew_ex_Farfetch'd": {'win_count': 2, 'lose_count': 0},
  'Greninja_Marshadow': {'win_count': 2, 'lose_count': 0},
  'Mew_ex_Alakazam': {'win_count': 2, 'lose_count': 0},
  'Moltres_ex_Rapidash': {'win_count': 2, 'lose_count': 0},
  'Marowak_ex_Aerodactyl_ex': {'win_count': 1, 'lose_count': 0},
  'Marowak_ex_Primeape': {'win_count': 2, 'lose_count': 0},
  'Marowak_ex_Golem': {'win_count': 1, 'lose_count': 0},
  "Golem_Farfetch'd": {'win_count': 1, 'lose_count': 0},
  'Machamp_ex_Mew_ex': {'win_count': 1, 'lose_count': 0},
  'Pidgeot_ex_Greninja': {'win_count': 1, 'lose_count': 0},
  'Mew_ex_Mewtwo_ex': {'win_count': 1, 'lo

## step a-2) 함수화
- 하나의 tourney에서 meta 정보를 불러온다.
- 기존의 tourney_data 에 정보를 누적시킨다.

In [74]:
# 전처리 함수 : 덱 타입
def preprocess_deck_type(deck_type):
    processed_deck_type = re.sub(r'[^a-zA-Z ]', '', deck_type)  # 영어 알파벳(대소문자)과 공백만 남기고 모두 제거
    return processed_deck_type.replace(' ', '_')  # 공백문자를 _로 변경

# 시간측정 데코레이터
def time_check(func):
  def wrapper(*args, **kwargs):
    start_time = datetime.now().replace(microsecond=0)  # 시작 시각
    print(f"함수 {func.__name__} 실행 시작 시간: {start_time}")
    
    result = func(*args, **kwargs) # 함수 실행

    end_time = datetime.now().replace(microsecond=0)  # 종료 시각
    print(f"함수 {func.__name__} 실행 종료 시간: {end_time}")
    print(f"함수 {func.__name__} 실행 시간: {end_time - start_time}")

    return result
  return wrapper

# tourney_data 에 1개의 tourney 메타 정보 누적
def crawl_one_tourney_meta(tourney_url, tourney_data, str_progress):
    print("[start] crawl_one_tourney_meta()")
    tourney_meta_url = tourney_url + "/metagame"
    soup = get_soup(tourney_meta_url)    # bs4로 html 가져오기
    tourney_deck_type_list = []
    min_deck_count = 1              # 최소 덱 타입 수

    # 정보 추출 : 덱 타입
    meta_tr_list = soup.select('table.meta tr')[1:]
    for tr in meta_tr_list:
        deck_count = int(tr.select('td')[1].text)
        if deck_count < min_deck_count:
            continue
        deck_type = preprocess_deck_type(tr.select('td')[2].text)
        deck_url = limitlesstcg_url_base + tr.select('td')[2].find('a')['href']
        # print(deck_count, deck_type, deck_url)
    
        # DB에 덱타입 추가 : 기존 정보가 있는지 확인
        if deck_type not in tourney_data:
            tourney_data[deck_type] = {}
        
        # df에 저장
        row = [deck_type, deck_count, deck_url]
        tourney_deck_type_list.append(row)
    
    # df : 덱 타입 (중복 덱타입 존재함)
    df_tourney_deck_type_list  = pd.DataFrame(tourney_deck_type_list, columns=["deck_type", "deck_count", "deck_url"])

    # 덱 타입 별 승/패 수 누적
    for index, row in df_tourney_deck_type_list.iterrows():
        print(f"Tourney {str_progress} Deck Type [{index+1}/{len(df_tourney_deck_type_list)}] {row['deck_type']}")
        matchup_deck_type = row['deck_type']
        matchup_url = row['deck_url'] + "/matchups"
        matchup_soup = get_soup(matchup_url)
    
        ### 정보 추출 : 덱 별 매치업 통계
        matchup_tr_list = matchup_soup.select('table.striped tr')[1:-1]
        
        for tr in matchup_tr_list:
            # 정보 추출
            opponent_deck_type = tr.select('td')[1].text.replace(' ', '_')
            win_count = int(tr.select('td')[2].text.split(' - ')[0])
            lose_count = int(tr.select('td')[2].text.split(' - ')[1])
            # print(opponent_deck_type, win_count, lose_count)
        
            # 정보 저장 : opponent_deck_type이 tourney_data[deck_type]에 있는지 확인 (없으면 추가)
            try:
                tourney_data[matchup_deck_type][opponent_deck_type]
            except: 
                tourney_data[matchup_deck_type][opponent_deck_type] = {"win_count": 0, "lose_count": 0}
        
            # 3. 승리/패배 횟수 누적
            tourney_data[matchup_deck_type][opponent_deck_type]["win_count"] += win_count
            tourney_data[matchup_deck_type][opponent_deck_type]["lose_count"] += lose_count
    
            # print(tourney_data[matchup_deck_type])
    
    print("[end] crawl_one_tourney_meta()")
    
    return tourney_data

## step b) 20장 덱 유형 별 vs 아키타입 승률

## step c) 아키타입 별 승률 높은 20장 덱 유형 분석 (궁극의 덱이 무엇일까)

# 2. tourney_result.csv 받아서 다수의 토너먼트에 대해 실행 + 파일로 저장

## 2-a. 2개의 tourney에 대해 테스트

In [90]:
### 파일 불러오기
tourney_result_file_name = "tourney_result_250125.csv"
df = pd.read_csv("./tourney_result/" + tourney_result_file_name, encoding='utf-8')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126 entries, 0 to 125
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   date          126 non-null    object
 1   tourney_name  126 non-null    object
 2   organizer     126 non-null    object
 3   players       126 non-null    int64 
 4   winner        126 non-null    object
 5   country       126 non-null    object
 6   url           126 non-null    object
dtypes: int64(1), object(6)
memory usage: 7.0+ KB


In [91]:
# copy 데이터프레임 만들고, 값 확인
df_tourney_result = df
df_tourney_result['url'][0]

'https://play.limitlesstcg.com/tournament/6782baab7960ec710eff019b'

In [92]:
# tourney_data 선언
tourney_data = {}

In [93]:
for index, row in df_tourney_result.iterrows():
    print(index, row['url'])
    tourney_data = crawl_one_tourney_meta(row['url'], tourney_data)

0 https://play.limitlesstcg.com/tournament/6782baab7960ec710eff019b


TypeError: crawl_one_tourney_meta() missing 1 required positional argument: 'str_progress'

In [94]:
tourney_data

{}

In [29]:
backup_t_data = tourney_data
backup_t_data

{'mewtwo_ex_gardevoir': {'Gyarados_ex_Greninja': {'win_count': 5,
   'lose_count': 12},
  'Charizard_ex_Moltres_ex': {'win_count': 8, 'lose_count': 6},
  'Pidgeot_Arbok': {'win_count': 2, 'lose_count': 0},
  'Arbok_Weezing': {'win_count': 1, 'lose_count': 0},
  'Gengar_ex_Gengar': {'win_count': 1, 'lose_count': 0},
  'Arcanine_ex_Moltres_ex': {'win_count': 1, 'lose_count': 0},
  'Articuno_ex': {'win_count': 1, 'lose_count': 0},
  'Gyarados_ex_Vaporeon': {'win_count': 1, 'lose_count': 0},
  'Serperior_Celebi_ex': {'win_count': 2, 'lose_count': 1},
  'Celebi_ex_Serperior': {'win_count': 8, 'lose_count': 7},
  "Aerodactyl_ex_Farfetch'd": {'win_count': 1, 'lose_count': 1},
  'Gyarados_ex_Articuno_ex': {'win_count': 1, 'lose_count': 1},
  'Exeggutor_ex_Mew_ex': {'win_count': 2, 'lose_count': 1},
  'Mewtwo_ex_Gardevoir': {'win_count': 18, 'lose_count': 18},
  'Greninja_Tauros': {'win_count': 1, 'lose_count': 3},
  'Exeggutor_ex_Celebi_ex': {'win_count': 10, 'lose_count': 14},
  'Pikachu_ex_Z

In [44]:
# JSON 파일로 저장 
meta_file_name = "backup_t_data.json"
with open("./tourney_meta/" + meta_file_name, "w") as f:
    json.dump(backup_t_data, f, indent=4)  # indent=4는 가독성을 위해 들여쓰기

In [45]:
# JSON 파일에서 2중 딕셔너리 로드

with open("./tourney_meta/" + meta_file_name, "r") as f:
    loaded_data = json.load(f)

print(loaded_data)

{'mewtwo_ex_gardevoir': {'Gyarados_ex_Greninja': {'win_count': 5, 'lose_count': 12}, 'Charizard_ex_Moltres_ex': {'win_count': 8, 'lose_count': 6}, 'Pidgeot_Arbok': {'win_count': 2, 'lose_count': 0}, 'Arbok_Weezing': {'win_count': 1, 'lose_count': 0}, 'Gengar_ex_Gengar': {'win_count': 1, 'lose_count': 0}, 'Arcanine_ex_Moltres_ex': {'win_count': 1, 'lose_count': 0}, 'Articuno_ex': {'win_count': 1, 'lose_count': 0}, 'Gyarados_ex_Vaporeon': {'win_count': 1, 'lose_count': 0}, 'Serperior_Celebi_ex': {'win_count': 2, 'lose_count': 1}, 'Celebi_ex_Serperior': {'win_count': 8, 'lose_count': 7}, "Aerodactyl_ex_Farfetch'd": {'win_count': 1, 'lose_count': 1}, 'Gyarados_ex_Articuno_ex': {'win_count': 1, 'lose_count': 1}, 'Exeggutor_ex_Mew_ex': {'win_count': 2, 'lose_count': 1}, 'Mewtwo_ex_Gardevoir': {'win_count': 18, 'lose_count': 18}, 'Greninja_Tauros': {'win_count': 1, 'lose_count': 3}, 'Exeggutor_ex_Celebi_ex': {'win_count': 10, 'lose_count': 14}, 'Pikachu_ex_Zebstrika': {'win_count': 10, 'lose_

In [39]:
type(loaded_data['mewtwo_ex_gardevoir']['Gyarados_ex_Greninja']['win_count'])

int

## 2-b. 함수화

In [95]:
@time_check
def crawl_tourney_meta(tourney_result_file_name):
    # 1. 파일 불러오기
    df = pd.read_csv("./tourney_result/" + tourney_result_file_name, encoding='utf-8')
    df.info()

    # tourney_meta 선언
    tourney_meta = {}

    # 2. 모든 url에 대해 meta 정보 수집
    for index, row in df.iterrows():
        str_progress = f"[{index+1}/{len(df)}]"
        print(f"Tourney Meta {str_progress} {row['url']}")
        tourney_meta = crawl_one_tourney_meta(row['url'], tourney_meta, str_progress)

    # JSON 파일로 저장 
    meta_file_name = tourney_result_file_name.replace('csv', 'json').replace('result', 'meta')
    with open("./tourney_meta/" + meta_file_name, "w") as f:
        json.dump(tourney_meta, f, indent=4)  # indent=4는 가독성을 위해 들여쓰기


In [58]:
tourney_result_file_name = "tourney_result_250123_131739.csv"
crawl_tourney_meta(tourney_result_file_name)


함수 crawl_tourney_meta 실행 시작 시간: 2025-01-23 18:03:55
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   date          2 non-null      object
 1   tourney_name  2 non-null      object
 2   organizer     2 non-null      object
 3   players       2 non-null      int64 
 4   winner        2 non-null      object
 5   country       2 non-null      object
 6   url           2 non-null      object
dtypes: int64(1), object(6)
memory usage: 240.0+ bytes
tourney_meta [1/2] https://play.limitlesstcg.com/tournament/678ae345010576265d112130
[start] crawl_one_tourney_meta()
16 mewtwo_ex_gardevoir https://play.limitlesstcg.com/tournament/678ae345010576265d112130/metagame/mewtwo-ex-gardevoir-a1
9 gyarados_ex_greninja https://play.limitlesstcg.com/tournament/678ae345010576265d112130/metagame/gyarados-ex-a1a-greninja-a1
6 pikachu_ex_zebstrika https://play.limitlesstcg.c

## 2-c. 모든 tourney에 대해 실행

In [118]:
### 파일 불러오기
tourney_result_file_name = "tourney_result_250125.csv"
df_result = pd.read_csv("./tourney_result/" + tourney_result_file_name, encoding='utf-8')
df_result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126 entries, 0 to 125
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   date          126 non-null    object
 1   tourney_name  126 non-null    object
 2   organizer     126 non-null    object
 3   players       126 non-null    int64 
 4   winner        126 non-null    object
 5   country       126 non-null    object
 6   url           126 non-null    object
dtypes: int64(1), object(6)
memory usage: 7.0+ KB


In [98]:
df_result.head(30)

,date,tourney_name,organizer,players,winner,country,url
0,2025-01-25,Pocket Turtle Weekly 9,Turtle Squad,157,ParaLouko,Brazil,https://play.limitlesstcg.com/tournament/6782b...
1,2025-01-25,Team Rocket Pocket Cup #29,Team Rocket,210,Tulkaz,Bolivia,https://play.limitlesstcg.com/tournament/6790c...
2,2025-01-24,Flash Friday #5,Pokemon Downunder,154,Rei 00,Japan,https://play.limitlesstcg.com/tournament/678dd...
3,2025-01-24,Torneio TD PADRÃO - 3#/Janeiro - Free - BR,Thiago Donola,143,HumanMader,Taiwan,https://play.limitlesstcg.com/tournament/678a5...
4,2025-01-24,FrogEX Weekly Cup #4 50 USD Prize,FrogEX,431,Riczinnn,Brazil,https://play.limitlesstcg.com/tournament/67892...
5,2025-01-24,Team Rocket Pocket Cup #27,Team Rocket,167,Polish,Japan,https://play.limitlesstcg.com/tournament/6790c...
6,2025-01-23,"IDEAVOLUTION Brawls 185,000.00",PTCGO-ID,334,Lowbattz,Indonesia,https://play.limitlesstcg.com/tournament/678bc...
7,2025-01-23,The Dark League Wednesday #4,The Dark League,184,Nana7x7,Japan,https://play.limitlesstcg.com/tournament/678d6...
8,2025-01-23,Pocket League #9,Mr-Gatito,180,ssEisen,Portugal,https://play.limitlesstcg.com/tournament/67885...
9,2025-01-22,Last Minute STANDARD FAST ROUNDS,Pokemon Downunder,163,Ekkinx,Spain,https://play.limitlesstcg.com/tournament/678f6...


In [119]:
tourney_result_file_name = "tourney_result_250125.csv"
crawl_tourney_meta(tourney_result_file_name)

함수 crawl_tourney_meta 실행 시작 시간: 2025-01-26 02:08:13
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126 entries, 0 to 125
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   date          126 non-null    object
 1   tourney_name  126 non-null    object
 2   organizer     126 non-null    object
 3   players       126 non-null    int64 
 4   winner        126 non-null    object
 5   country       126 non-null    object
 6   url           126 non-null    object
dtypes: int64(1), object(6)
memory usage: 7.0+ KB
Tourney Meta [1/126] https://play.limitlesstcg.com/tournament/6782baab7960ec710eff019b
[start] crawl_one_tourney_meta()
Tourney [1/126] Deck Type [1/54] Mewtwo_ex_Gardevoir
Tourney [1/126] Deck Type [2/54] Gyarados_ex_Greninja
Tourney [1/126] Deck Type [3/54] Exeggutor_ex_Celebi_ex
Tourney [1/126] Deck Type [4/54] Arcanine_ex_Moltres_ex
Tourney [1/126] Deck Type [5/54] Pikachu_ex_Zebstrika
Tourney [1/126] Deck Ty

In [120]:
# JSON 파일에서 2중 딕셔너리 로드
tourney_result_file_name = "tourney_result_250125.csv"
meta_file_name = tourney_result_file_name.replace('csv', 'json').replace('result', 'meta')
with open("./tourney_meta/" + meta_file_name, "r") as f:
    loaded_meta_data = json.load(f)

print(loaded_meta_data)

{'Mewtwo_ex_Gardevoir': {'Wigglytuff_ex_Alakazam': {'win_count': 1, 'lose_count': 0}, 'Machamp_ex_Dugtrio': {'win_count': 7, 'lose_count': 8}, 'Exeggutor_ex_Exeggutor': {'win_count': 34, 'lose_count': 31}, 'Mew_ex_Melmetal': {'win_count': 19, 'lose_count': 9}, 'Aerodactyl_ex_Pidgeot': {'win_count': 5, 'lose_count': 5}, 'Dragonite_Druddigon': {'win_count': 37, 'lose_count': 13}, 'Gyarados_ex_Chatot': {'win_count': 24, 'lose_count': 22}, 'Charizard_ex_Arcanine_ex': {'win_count': 561, 'lose_count': 355}, 'Blastoise_ex_Articuno_ex': {'win_count': 26, 'lose_count': 19}, 'Starmie_ex_Lumineon': {'win_count': 109, 'lose_count': 88}, 'Charizard_ex_Moltres_ex': {'win_count': 607, 'lose_count': 448}, 'Greninja_Tauros': {'win_count': 84, 'lose_count': 91}, 'Pikachu_ex_Zebstrika': {'win_count': 1771, 'lose_count': 1390}, 'Mewtwo_ex_Gardevoir': {'win_count': 3281, 'lose_count': 3281}, 'Aerodactyl_ex_Primeape': {'win_count': 609, 'lose_count': 287}, 'Greninja_Farfetchd': {'win_count': 90, 'lose_count

In [121]:
def calculate_wins_losses_from_meta(meta_data):
    total_wins = 0
    total_losses = 0
    for deck_type, matchups in meta_data.items():
        for opponent_deck, results in matchups.items():
            total_wins += results["win_count"]
            total_losses += results["lose_count"]
    return total_wins, total_losses

total_wins, total_losses = calculate_wins_losses_from_meta(loaded_meta_data)
total_wins, total_losses  # 기권패를 제외한 통계라, loss가 적음

(93686, 88384)

In [122]:
# 총 88047 건의 매치
(total_wins + total_losses)/2

91035.0

In [123]:
def make_df_from_meta(meta_data):
    temp_list = []
    for deck_type, matchups in meta_data.items():
        win_count = 0
        loss_count = 0
        for opponent_deck, results in matchups.items():
            win_count += results["win_count"]
            loss_count += results["lose_count"]

        matchup_count = win_count + loss_count 
        if matchup_count == 0:
            continue
        else:
            win_rate = win_count / (win_count + loss_count)
            temp_row = [deck_type, matchup_count, win_count, loss_count, win_rate]
            temp_list.append(temp_row)
        
    df = pd.DataFrame(temp_list, columns=["deck_type", "matchup_count", "win_count", "loss_count", "win_rate"])
    return df

df_meta = make_df_from_meta(loaded_meta_data)
df_meta

,deck_type,matchup_count,win_count,loss_count,win_rate
0,Mewtwo_ex_Gardevoir,33920,17351,16569,0.511527
1,Gyarados_ex_Greninja,23697,12715,10982,0.536566
2,Exeggutor_ex_Celebi_ex,9668,5273,4395,0.545408
3,Arcanine_ex_Moltres_ex,4498,2395,2103,0.532459
4,Pikachu_ex_Zebstrika,17309,9177,8132,0.530187
...,...,...,...,...,...
803,Sandslash_Dugtrio,6,4,2,0.666667
804,Sandslash_Grapploct,2,0,2,0.000000
805,Articuno_ex_Moltres_ex,3,1,2,0.333333
806,Starmie_ex_Pidgeot,4,2,2,0.500000


In [124]:
# 덱 타입 중복 없음
df_meta['deck_type'].nunique()

808

In [125]:
df_meta.head(30)

,deck_type,matchup_count,win_count,loss_count,win_rate
0,Mewtwo_ex_Gardevoir,33920,17351,16569,0.511527
1,Gyarados_ex_Greninja,23697,12715,10982,0.536566
2,Exeggutor_ex_Celebi_ex,9668,5273,4395,0.545408
3,Arcanine_ex_Moltres_ex,4498,2395,2103,0.532459
4,Pikachu_ex_Zebstrika,17309,9177,8132,0.530187
5,Charizard_ex_Moltres_ex,6006,2981,3025,0.496337
6,Celebi_ex_Serperior,10328,5033,5295,0.487316
7,Exeggutor_ex_Mew_ex,1616,855,761,0.529084
8,Pikachu_ex_Raichu,5318,2849,2469,0.535728
9,Greninja_Tauros,918,498,420,0.542484


In [126]:
df_meta_sorted = df_meta.sort_values(by='win_rate', ascending=False)
df_meta_sorted.head(100)

,deck_type,matchup_count,win_count,loss_count,win_rate
645,Charizard_ex_Greninja,1,1,0,1.000000
638,Aerodactyl_ex_Alakazam,3,3,0,1.000000
388,Charizard_ex_Moltres,2,2,0,1.000000
380,Dragonite_Articuno_ex,1,1,0,1.000000
756,Greninja_Lapras_ex,2,2,0,1.000000
...,...,...,...,...,...
411,Greninja_Raichu,6,4,2,0.666667
231,Exeggutor_ex_Venusaur,3,2,1,0.666667
140,Machamp_ex_Farfetchd,29,19,10,0.655172
550,Victreebel_Exeggutor_ex,23,15,8,0.652174


In [127]:
df_meta_sorted = df_meta.sort_values(by='matchup_count', ascending=False)
df_meta_top20 = df_meta_sorted.head(21)
df_meta_top20

,deck_type,matchup_count,win_count,loss_count,win_rate
0,Mewtwo_ex_Gardevoir,33920,17351,16569,0.511527
1,Gyarados_ex_Greninja,23697,12715,10982,0.536566
4,Pikachu_ex_Zebstrika,17309,9177,8132,0.530187
6,Celebi_ex_Serperior,10328,5033,5295,0.487316
2,Exeggutor_ex_Celebi_ex,9668,5273,4395,0.545408
5,Charizard_ex_Moltres_ex,6006,2981,3025,0.496337
8,Pikachu_ex_Raichu,5318,2849,2469,0.535728
16,Aerodactyl_ex_Primeape,5124,2701,2423,0.527127
38,Charizard_ex_Arcanine_ex,4987,2661,2326,0.533587
12,Scolipede_Weezing,4686,2188,2498,0.466923


In [128]:
df_meta_top20 = df_meta_top20[df_meta_top20['deck_type'] != 'Pikachu_ex_Zapdos_ex'].reset_index(drop=True)
df_meta_top20

,deck_type,matchup_count,win_count,loss_count,win_rate
0,Mewtwo_ex_Gardevoir,33920,17351,16569,0.511527
1,Gyarados_ex_Greninja,23697,12715,10982,0.536566
2,Pikachu_ex_Zebstrika,17309,9177,8132,0.530187
3,Celebi_ex_Serperior,10328,5033,5295,0.487316
4,Exeggutor_ex_Celebi_ex,9668,5273,4395,0.545408
5,Charizard_ex_Moltres_ex,6006,2981,3025,0.496337
6,Pikachu_ex_Raichu,5318,2849,2469,0.535728
7,Aerodactyl_ex_Primeape,5124,2701,2423,0.527127
8,Charizard_ex_Arcanine_ex,4987,2661,2326,0.533587
9,Scolipede_Weezing,4686,2188,2498,0.466923


In [129]:
top20_deck_list = df_meta_top20['deck_type'].to_list()
top20_deck_list

['Mewtwo_ex_Gardevoir',
 'Gyarados_ex_Greninja',
 'Pikachu_ex_Zebstrika',
 'Celebi_ex_Serperior',
 'Exeggutor_ex_Celebi_ex',
 'Charizard_ex_Moltres_ex',
 'Pikachu_ex_Raichu',
 'Aerodactyl_ex_Primeape',
 'Charizard_ex_Arcanine_ex',
 'Scolipede_Weezing',
 'Arcanine_ex_Moltres_ex',
 'Golem_Druddigon',
 'Greninja_Mew_ex',
 'Aerodactyl_ex_Farfetchd',
 'Ninetales_Rapidash',
 'Starmie_ex_Articuno_ex',
 'Exeggutor_ex_Mew_ex',
 'Pidgeot_ex_Pidgeot',
 'Starmie_ex_Gyarados_ex',
 'Aerodactyl_ex_Mew_ex']

In [130]:
def create_winrate_df(data_dict, deck_list):
    df = pd.DataFrame(index=deck_list, columns=deck_list)

    for deck1 in deck_list:
        for deck2 in deck_list:
            if deck1 == deck2:
                df.loc[deck1, deck2] = ''  # 자기 자신과의 대결은 빈칸
            elif deck2 in data_dict.get(deck1, {}):
                win_count = data_dict[deck1][deck2]['win_count']
                lose_count = data_dict[deck1][deck2]['lose_count']
                total_matches = win_count + lose_count
                win_rate = (win_count / total_matches) * 100 if total_matches > 0 else float('nan')
                df.loc[deck1, deck2] = f"{win_rate:.2f}%"  # 승률을 백분율로 표시
            else:
                df.loc[deck1, deck2] = ''  # 상대 전적이 없으면 빈칸

    return df

# DataFrame 생성
df_meta_stat = create_winrate_df(loaded_meta_data, top20_deck_list)

# 결과 출력
df_meta_stat

,Mewtwo_ex_Gardevoir,Gyarados_ex_Greninja,Pikachu_ex_Zebstrika,Celebi_ex_Serperior,Exeggutor_ex_Celebi_ex,Charizard_ex_Moltres_ex,Pikachu_ex_Raichu,Aerodactyl_ex_Primeape,Charizard_ex_Arcanine_ex,Scolipede_Weezing,Arcanine_ex_Moltres_ex,Golem_Druddigon,Greninja_Mew_ex,Aerodactyl_ex_Farfetchd,Ninetales_Rapidash,Starmie_ex_Articuno_ex,Exeggutor_ex_Mew_ex,Pidgeot_ex_Pidgeot,Starmie_ex_Gyarados_ex,Aerodactyl_ex_Mew_ex
Mewtwo_ex_Gardevoir,,33.40%,56.03%,58.77%,50.46%,57.54%,51.30%,67.97%,61.24%,31.93%,48.44%,61.97%,43.98%,59.19%,57.75%,56.97%,44.09%,50.22%,54.84%,59.84%
Gyarados_ex_Greninja,66.58%,,24.68%,62.80%,60.45%,55.32%,35.39%,36.03%,50.45%,70.41%,76.45%,52.57%,50.92%,27.94%,66.20%,58.13%,67.32%,60.00%,59.34%,36.78%
Pikachu_ex_Zebstrika,43.97%,75.24%,,40.31%,33.62%,55.98%,47.63%,35.81%,48.01%,60.78%,39.13%,44.52%,56.47%,51.31%,57.14%,70.90%,35.29%,65.89%,74.00%,47.30%
Celebi_ex_Serperior,41.23%,37.18%,59.69%,,44.92%,43.79%,49.12%,59.69%,24.47%,57.27%,25.48%,59.24%,54.26%,62.75%,29.93%,46.55%,61.36%,61.43%,49.35%,53.33%
Exeggutor_ex_Celebi_ex,49.57%,39.48%,66.38%,55.24%,,38.44%,65.74%,69.62%,27.85%,57.46%,35.34%,72.85%,60.69%,69.50%,44.93%,45.35%,71.43%,64.06%,50.82%,64.37%
Charizard_ex_Moltres_ex,42.50%,44.77%,44.09%,56.21%,61.68%,,43.80%,38.42%,47.88%,64.91%,58.94%,77.78%,34.02%,44.07%,55.74%,40.00%,56.34%,62.79%,58.06%,41.46%
Pikachu_ex_Raichu,48.81%,64.77%,52.60%,51.06%,34.72%,57.78%,,31.50%,48.51%,58.74%,42.86%,45.65%,52.46%,29.79%,45.16%,61.40%,45.83%,70.97%,75.00%,20.00%
Aerodactyl_ex_Primeape,32.28%,64.26%,64.92%,41.11%,31.14%,62.90%,70.63%,,56.41%,76.47%,44.33%,62.24%,62.35%,39.29%,67.44%,57.14%,40.82%,66.67%,45.45%,33.33%
Charizard_ex_Arcanine_ex,38.93%,49.78%,52.18%,75.95%,72.52%,52.73%,53.33%,44.87%,,51.28%,58.62%,72.53%,47.30%,55.71%,64.10%,43.90%,56.10%,54.84%,51.11%,38.24%
Scolipede_Weezing,68.22%,29.75%,39.47%,43.02%,43.85%,37.84%,42.66%,25.00%,48.72%,,39.04%,20.83%,51.52%,11.54%,38.71%,42.31%,47.37%,43.75%,34.88%,31.58%


In [132]:
# 파일로 저장
df_meta_stat.to_csv("tourney_stat_250125.csv", encoding="utf-8")  # 엑셀로 저장

In [69]:
type(loaded_meta_data)

dict

In [70]:
type(top20_deck_list)

list

In [75]:
import pandas as pd

def create_winrate_df(data_dict, deck_list):
    df = pd.DataFrame(index=deck_list, columns=deck_list)

    for deck1 in deck_list:
        for deck2 in deck_list:
            if deck1 == deck2:
                df.loc[deck1, deck2] = ''  # 자기 자신과의 대결은 빈칸
            elif deck2 in data_dict.get(deck1, {}):
                # data_dict에서 win_count와 lose_count 가져오기 전에 확인
                print(f"--- {deck1} vs {deck2} ---")
                print(f"  data_dict.get({deck1}, {{}}): {data_dict.get(deck1, {})}")
                print(f"  {deck2} in data_dict.get({deck1}, {{}}): {deck2 in data_dict.get(deck1, {})}")

                if deck2 in data_dict.get(deck1, {}):  # 이 조건문을 추가
                    win_count = data_dict[deck1][deck2]['win_count']
                    lose_count = data_dict[deck1][deck2]['lose_count']
                    print(f"  win_count: {win_count}, lose_count: {lose_count}")
                    total_matches = win_count + lose_count
                    win_rate = (win_count / total_matches) * 100 if total_matches > 0 else float('nan')
                    df.loc[deck1, deck2] = f"{win_rate:.2f}%"  # 승률을 백분율로 표시
                else:
                    print(f"  {deck2} not found in data_dict for {deck1}")
            else:
                df.loc[deck1, deck2] = ''  # 상대 전적이 없으면 빈칸

    return df

# 예시 데이터 (일부만 사용)
data_dict = {
    'mewtwo_ex_gardevoir': {
        'gyarados_ex_greninja': {'win_count': 5, 'lose_count': 12},
        'charizard_ex_moltres_ex': {'win_count': 8, 'lose_count': 6},
    },
    'gyarados_ex_greninja': {
        'mewtwo_ex_gardevoir': {'win_count': 12, 'lose_count': 5},
        'charizard_ex_moltres_ex': {'win_count': 4, 'lose_count': 4},
    },
}
deck_list = ['mewtwo_ex_gardevoir', 'gyarados_ex_greninja', 'charizard_ex_moltres_ex']

# DataFrame 생성
df = create_winrate_df(data_dict, deck_list)

# 결과 출력
df

--- mewtwo_ex_gardevoir vs gyarados_ex_greninja ---
  data_dict.get(mewtwo_ex_gardevoir, {}): {'gyarados_ex_greninja': {'win_count': 5, 'lose_count': 12}, 'charizard_ex_moltres_ex': {'win_count': 8, 'lose_count': 6}}
  gyarados_ex_greninja in data_dict.get(mewtwo_ex_gardevoir, {}): True
  win_count: 5, lose_count: 12
--- mewtwo_ex_gardevoir vs charizard_ex_moltres_ex ---
  data_dict.get(mewtwo_ex_gardevoir, {}): {'gyarados_ex_greninja': {'win_count': 5, 'lose_count': 12}, 'charizard_ex_moltres_ex': {'win_count': 8, 'lose_count': 6}}
  charizard_ex_moltres_ex in data_dict.get(mewtwo_ex_gardevoir, {}): True
  win_count: 8, lose_count: 6
--- gyarados_ex_greninja vs mewtwo_ex_gardevoir ---
  data_dict.get(gyarados_ex_greninja, {}): {'mewtwo_ex_gardevoir': {'win_count': 12, 'lose_count': 5}, 'charizard_ex_moltres_ex': {'win_count': 4, 'lose_count': 4}}
  mewtwo_ex_gardevoir in data_dict.get(gyarados_ex_greninja, {}): True
  win_count: 12, lose_count: 5
--- gyarados_ex_greninja vs charizard_

,mewtwo_ex_gardevoir,gyarados_ex_greninja,charizard_ex_moltres_ex
mewtwo_ex_gardevoir,,29.41%,57.14%
gyarados_ex_greninja,70.59%,,50.00%
charizard_ex_moltres_ex,,,


In [76]:
# DataFrame 생성
df = create_winrate_df(loaded_meta_data, top20_deck_list)

# 결과 출력
df

,mewtwo_ex_gardevoir,gyarados_ex_greninja,pikachu_ex_zebstrika,celebi_ex_serperior,exeggutor_ex_celebi_ex,charizard_ex_moltres_ex,pikachu_ex_raichu,aerodactyl_ex_primeape,charizard_ex_arcanine_ex,scolipede_weezing,arcanine_ex_moltres_ex,golem_druddigon,greninja_mew_ex,aerodactyl_ex_farfetchd,ninetales_rapidash,starmie_ex_articuno_ex,exeggutor_ex_mew_ex,pidgeot_ex_pidgeot,starmie_ex_gyarados_ex,aerodactyl_ex_mew_ex
mewtwo_ex_gardevoir,,,,,,,,,,,,,,,,,,,,
gyarados_ex_greninja,,,,,,,,,,,,,,,,,,,,
pikachu_ex_zebstrika,,,,,,,,,,,,,,,,,,,,
celebi_ex_serperior,,,,,,,,,,,,,,,,,,,,
exeggutor_ex_celebi_ex,,,,,,,,,,,,,,,,,,,,
charizard_ex_moltres_ex,,,,,,,,,,,,,,,,,,,,
pikachu_ex_raichu,,,,,,,,,,,,,,,,,,,,
aerodactyl_ex_primeape,,,,,,,,,,,,,,,,,,,,
charizard_ex_arcanine_ex,,,,,,,,,,,,,,,,,,,,
scolipede_weezing,,,,,,,,,,,,,,,,,,,,


In [79]:
loaded_meta_data['mewtwo_ex_gardevoir']

{'Charizard_ex_Arcanine_ex': {'win_count': 538, 'lose_count': 344},
 'Greninja_Hitmonlee': {'win_count': 7, 'lose_count': 5},
 'Muk_Weezing': {'win_count': 11, 'lose_count': 10},
 'Machamp_ex_Aerodactyl_ex': {'win_count': 36, 'lose_count': 6},
 'Alakazam_Jynx': {'win_count': 5, 'lose_count': 0},
 'Moltres_ex_Centiskorch': {'win_count': 15, 'lose_count': 9},
 'Greninja_Gyarados_ex': {'win_count': 35, 'lose_count': 55},
 'Starmie_ex_Articuno_ex': {'win_count': 182, 'lose_count': 134},
 'Starmie_ex_Lumineon': {'win_count': 105, 'lose_count': 83},
 'Aerodactyl_ex_Primeape': {'win_count': 599, 'lose_count': 278},
 "Starmie_ex_Farfetch'd": {'win_count': 1, 'lose_count': 0},
 'Machamp_ex': {'win_count': 16, 'lose_count': 5},
 "Greninja_Farfetch'd": {'win_count': 79, 'lose_count': 74},
 'Exeggutor_ex_Mew_ex': {'win_count': 129, 'lose_count': 157},
 'Pikachu_ex_Raichu': {'win_count': 466, 'lose_count': 438},
 'Greninja_Mew_ex': {'win_count': 189, 'lose_count': 249},
 'Pikachu_ex_Zebstrika': {'w

#### 샘플

In [84]:
### 샘플
tourney_result_sample_file_name = "tourney_result_sample.csv"
df_sample = pd.read_csv("./tourney_result/" + tourney_result_file_name, encoding='utf-8')
crawl_tourney_meta(tourney_result_sample_file_name)

함수 crawl_tourney_meta 실행 시작 시간: 2025-01-25 21:55:32
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   date          2 non-null      object
 1   tourney_name  2 non-null      object
 2   organizer     2 non-null      object
 3   players       2 non-null      int64 
 4   winner        2 non-null      object
 5   country       2 non-null      object
 6   url           2 non-null      object
dtypes: int64(1), object(6)
memory usage: 240.0+ bytes
Tourney Meta [1/2] https://play.limitlesstcg.com/tournament/678ae345010576265d112130
[start] crawl_one_tourney_meta()
Tourney [1/2] Deck Type [1/34] Mewtwo_ex_Gardevoir
Tourney [1/2] Deck Type [2/34] Gyarados_ex_Greninja
Tourney [1/2] Deck Type [3/34] Pikachu_ex_Zebstrika
Tourney [1/2] Deck Type [4/34] Exeggutor_ex_Celebi_ex
Tourney [1/2] Deck Type [5/34] Celebi_ex_Serperior
Tourney [1/2] Deck Type [6/34] Aeroda

In [86]:
# JSON 파일에서 2중 딕셔너리 로드
tourney_result_sample_file_name = "tourney_result_sample.csv"
meta_sample_file_name = tourney_result_sample_file_name.replace('csv', 'json').replace('result', 'meta')
with open("./tourney_meta/" + meta_sample_file_name, "r") as f:
    loaded_meta_sample_data = json.load(f)

print(loaded_meta_sample_data)

{'Mewtwo_ex_Gardevoir': {'Gyarados_ex_Greninja': {'win_count': 5, 'lose_count': 12}, 'Charizard_ex_Moltres_ex': {'win_count': 8, 'lose_count': 6}, 'Pidgeot_Arbok': {'win_count': 2, 'lose_count': 0}, 'Arbok_Weezing': {'win_count': 1, 'lose_count': 0}, 'Gengar_ex_Gengar': {'win_count': 1, 'lose_count': 0}, 'Arcanine_ex_Moltres_ex': {'win_count': 1, 'lose_count': 0}, 'Articuno_ex': {'win_count': 1, 'lose_count': 0}, 'Gyarados_ex_Vaporeon': {'win_count': 1, 'lose_count': 0}, 'Serperior_Celebi_ex': {'win_count': 2, 'lose_count': 1}, 'Celebi_ex_Serperior': {'win_count': 8, 'lose_count': 7}, "Aerodactyl_ex_Farfetch'd": {'win_count': 1, 'lose_count': 1}, 'Gyarados_ex_Articuno_ex': {'win_count': 1, 'lose_count': 1}, 'Exeggutor_ex_Mew_ex': {'win_count': 2, 'lose_count': 1}, 'Mewtwo_ex_Gardevoir': {'win_count': 18, 'lose_count': 18}, 'Greninja_Tauros': {'win_count': 1, 'lose_count': 3}, 'Exeggutor_ex_Celebi_ex': {'win_count': 10, 'lose_count': 14}, 'Pikachu_ex_Zebstrika': {'win_count': 10, 'lose_

In [88]:
# DataFrame 생성
sample_deck_list = ['Mewtwo_ex_Gardevoir', 'Gyarados_ex_Greninja']
df_sample_stat = create_winrate_df(loaded_meta_sample_data, sample_deck_list)

# 결과 출력
df_sample_stat

--- Mewtwo_ex_Gardevoir vs Gyarados_ex_Greninja ---
  data_dict.get(Mewtwo_ex_Gardevoir, {}): {'Gyarados_ex_Greninja': {'win_count': 5, 'lose_count': 12}, 'Charizard_ex_Moltres_ex': {'win_count': 8, 'lose_count': 6}, 'Pidgeot_Arbok': {'win_count': 2, 'lose_count': 0}, 'Arbok_Weezing': {'win_count': 1, 'lose_count': 0}, 'Gengar_ex_Gengar': {'win_count': 1, 'lose_count': 0}, 'Arcanine_ex_Moltres_ex': {'win_count': 1, 'lose_count': 0}, 'Articuno_ex': {'win_count': 1, 'lose_count': 0}, 'Gyarados_ex_Vaporeon': {'win_count': 1, 'lose_count': 0}, 'Serperior_Celebi_ex': {'win_count': 2, 'lose_count': 1}, 'Celebi_ex_Serperior': {'win_count': 8, 'lose_count': 7}, "Aerodactyl_ex_Farfetch'd": {'win_count': 1, 'lose_count': 1}, 'Gyarados_ex_Articuno_ex': {'win_count': 1, 'lose_count': 1}, 'Exeggutor_ex_Mew_ex': {'win_count': 2, 'lose_count': 1}, 'Mewtwo_ex_Gardevoir': {'win_count': 18, 'lose_count': 18}, 'Greninja_Tauros': {'win_count': 1, 'lose_count': 3}, 'Exeggutor_ex_Celebi_ex': {'win_count': 1

,Mewtwo_ex_Gardevoir,Gyarados_ex_Greninja
Mewtwo_ex_Gardevoir,,29.41%
Gyarados_ex_Greninja,70.59%,
